In [1]:
! pip3 install -U spacy -q 

In [2]:
!python3 -m spacy info

/Users/closerlook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

============================== Info about spaCy ==============================

spaCy version    3.7.3                         
Location         /Users/closerlook/Library/Python/3.9/lib/python/site-packages/spacy
Platform         macOS-14.3-x86_64-i386-64bit  
Python version   3.9.6                         
Pipelines        en_core_web_sm (3.7.1), en_core_web_md (3.7.1), en_core_web_lg (3.7.1), en_core_web_trf (3.7.3)



In [3]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

/Users/closerlook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/closerlook/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
f = open('annotations-8.json')
TRAIN_DATA = json.load(f)

In [3]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 28/28 [00:00<00:00, 1644.73it/s]


In [4]:
! python3 -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

/Users/closerlook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [5]:
! python3 -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

/Users/closerlook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.83    0.00    0.00    0.00    0.00
 39     200         95.95   2379.24  100.00  100.00  100.00    1.00
 89     400          0.00      0.00  100.00  100.00  100.00    1.00
152     600          0.00      0.00  100.00  100.00  100.00    1.00
220     800          0.00      

In [6]:
nlp_ner = spacy.load("model-best") 

In [9]:
doc = nlp_ner('''Make a trip to Downtown Louisville  ''') # input sample text

In [10]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter